# Learning Objectives

- Understand data acquisition: downloading from static links
- Understand data curation: working with hierarchically structured data (JSON)

## Where am I?

In [ ]:
getwd()

In [ ]:
setwd("/home/lngo/data-mining-r/")

In [ ]:
getwd()

## Data Curation

- For JSON and XML formats, we will use the [NY Retailed Food Store Database from data.gov](https://catalog.data.gov/dataset/retail-food-stores/resource/498a7e81-ea0e-425c-bb8d-a4e36d619f81)


### 1. JSON Format

- Data are stored in Key:Value format
- Example of data in JSON format (http://www.json.org/example.html)
```
{"menu": {
  "id": "file",
  "value": "File",
  "popup": {
    "menuitem": [
      {"value": "New", "onclick": "CreateNewDoc()"},
      {"value": "Open", "onclick": "OpenDoc()"},
      {"value": "Close", "onclick": "CloseDoc()"}
    ]
  }
}}
```

Package `RJSONIO` converst json data into nested lists. 

In [ ]:
library(RJSONIO)

In [ ]:
sample_json <- fromJSON('
{"menu": {
  "id": "file",
  "value": "File",
  "popup": {
    "menuitem": [
      {"value": "New", "onclick": "CreateNewDoc()"},
      {"value": "Open", "onclick": "OpenDoc()"},
      {"value": "Close", "onclick": "CloseDoc()"}
    ]
  }
}}
')

In [ ]:
str(sample_json)

- For simple data, the resulting converted data will consist of multiple nested lists. Each individual list within this collection often has name representing its KEY and the content presenting its VALUE, which in turn could be another list of vector of data. 

- Complex data in json format can lead to nested lists with deep hierarchy. One possible approach is to rely on `summary` and `str` functions of R to iteratively explore the hierarchical structure of the data

The JSON data can be read directly from an online location

In [ ]:
stores_json <- fromJSON('https://data.ny.gov/api/views/9a8c-vfzj/rows.json?accessType=DOWNLOAD')

JSON data can also be read in from a local file

In [ ]:
stores_json <- fromJSON('./samples/food_stores.json')

`summary` and `str` need to be used concurrently. `str` provides detailed information but could be too lengthy. `summary` provides a better overview of all data elements at a cost of missing internal details.

In [ ]:
str(stores_json)

In [ ]:
summary(stores_json)

It looks like this JSON data contains two main components: metadata and data. 

In [ ]:
str(stores_json[['meta']])

In [ ]:
summary(stores_json[['meta']])

Be very careful when you use `summary` or `str` on a list of data elements, as they will attemp to print out information on **every data element**. 

In [ ]:
stores_counts <- length(stores_json[['data']])
print (stores_counts)

In [ ]:
str(stores_json[['data']][[1]])

In [ ]:
summary(stores_json[['data']][[1]])

For this particular dataset, since the header information is separated from the data (`metadata` and `data`), it is reasonable to assume that all data elements follow the same structural rules, and that while there could be missing values, there will be no extra values

In cases where this assumption is not true, a possible quick method to check for discrepancies can be done as follows:

In [ ]:
column_count <- numeric(stores_counts)

for (i in 1:stores_counts){
    column_count[i] <- length(stores_json[['data']][[i]])
}

print (summary(column_count))
print (as.data.frame(table(unique(column_count))))

Depending on the results of `summary`, we can determine whether discrepancies exist. With the combination of `as.data.frame`, `table`, `unique`, we can determine the level of discrepancies and devise additional identification and unification solutions.  

At this time, we will focus on `metadata` to understand the structure of our data

In [ ]:
str(stores_json[['meta']][['view']])

In [ ]:
summary(stores_json[['meta']][['view']])

In [ ]:
str(stores_json[['meta']][['view']][['columns']])

In [ ]:
summary(stores_json[['meta']][['view']][['columns']])

In [ ]:
for (i in 1:23){
    tmpList <- stores_json[['meta']][['view']][['columns']][[i]]
    print (paste(i,tmpList[['name']],sep=".: "))
    print (names(tmpList))
}

In [ ]:
str(stores_json[['meta']][['view']][['columns']][[23]])

In [ ]:
summary(stores_json[['meta']][['view']][['columns']][[23]])

In [ ]:
str(stores_json[['meta']][['view']][['columns']][[23]][['subColumnTypes']])

In [ ]:
unique(stores_json[['meta']][['view']][['columns']][[23]][['subColumnTypes']])

In [ ]:
stores_json[['data']][[1]]

Time to build the data frame:
- Fields 1 to 22 are flattened, field 23 is not. What to do?
- Fortunately, field 23 is only nested at one level, and could be flattended into additioanl fields. In fact, you could also argue that subfield A is not relevant since it is covered by fields 15 through 21. 

In [ ]:
for (i in 1:23){
    tmpList <- stores_json[['meta']][['view']][['columns']][[i]]
    print (paste(i,tmpList[['name']],sep=".: "))
    if (i == 23){
        print (tmpList[['subColumnTypes']])
    }
}

In [ ]:
stores_headers <- c()

for (i in 1:23){  
    tmpList <- stores_json[['meta']][['view']][['columns']][[i]]
    if (i != 23){
        stores_headers <- c(stores_headers, tmpList[['name']])
    }
    else {
        for (j in 2:5){
            stores_headers <- c(stores_headers, tmpList[['subColumnTypes']][j])
        }
    }
}

print (stores_headers)

Since columns 1 through 8 seem to be related to the creation of the JSON data rather than the data themselves, it is OK to drop them in this case. 

In [ ]:
stores_headers <- c()

for (i in 1:23){  
    tmpList <- stores_json[['meta']][['view']][['columns']][[i]]
    if (i != 23){
        stores_headers <- c(stores_headers, tmpList[['name']])
    }
    else {
        for (j in 2:5){
            stores_headers <- c(stores_headers, tmpList[['subColumnTypes']][j])
        }
    }
}

print (stores_headers)

- Now we need to create the dataframe ...
- For complex nested lists, faster techniques using `dplyr`, `plyr`, or `reshape` might not work well due to data uncertainy ...

In [ ]:
df_stores <- data.frame(character(stores_counts), stringsAsFactors=FALSE)
for (i in 2:14){
    df_stores[,i] <- character(stores_counts)
}

for (i in 15:16){
    df_stores[,i] <- numeric(stores_counts)
}

for (i in 17:18){
    df_stores[,i] <- character(stores_counts)
}
colnames(df_stores) <- stores_headers

In [ ]:
str(df_stores)

In [ ]:
for (i in 1:1){
    tmpList <- stores_json[['data']][[i]]
    print (tmpList)
}

In [ ]:
for (i in 1:1){
    tmpList <- stores_json[['data']][[i]]
    for (j in 9:22){
        df_stores[i,j-8] <- tmpList[[j]]
    }
    for (j in 15:16){
        df_stores[i,j] <- as.numeric(tmpList[[23]][[j-13]])
    }
    for (j in 17:18){
        df_stores[i,j] <- tmpList[[23]][[j-13]]
    }
    print (df_stores[i,])
}

This looks correct ...

In [ ]:
for (i in 1:stores_counts){
    tmpList <- stores_json[['data']][[i]]
    for (j in 9:22){
        df_stores[i,j-8] <- tmpList[[j]]
    }
    for (j in 15:16){
        df_stores[i,j] <- as.numeric(tmpList[[23]][[j-13]])
    }
    for (j in 17:18){
        df_stores[i,j] <- tmpList[[23]][[j-13]]
    }
}

It doesn't ...

In [ ]:
print (i)
print (j)
print (tmpList[[j]])

Let's try this again ...

In [ ]:
for (i in 1:stores_counts){
    tmpList <- stores_json[['data']][[i]]
    for (j in 9:22){
        if (!is.null(tmpList[[j]])){
            df_stores[i,j-8] <- tmpList[[j]]
        }
    }
    for (j in 15:16){
        if (!is.null(tmpList[[23]][[j-13]])){
            df_stores[i,j] <- as.numeric(tmpList[[23]][[j-13]])
        }
    }
    for (j in 17:18){
        if (!is.null(tmpList[[23]][[j-13]])){
            df_stores[i,j] <- tmpList[[23]][[j-13]]
        }
    }
}

Validate data (to some extent ...)

In [ ]:
unique(df_stores$County)

In [ ]:
print(stores_json[['data']][10])
print(df_stores[10,])